## PROJECT - SCRAPING

In [1]:
!pip install plotly
!jupyter labextension install jupyterlab-plotly@5.2.1 @jupyter-widgets/jupyterlab-manager

     |████████████████████████████████| 23.9 MB 9.8 MB/s eta 0:00:01
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)
Building jupyterlab assets (build:prod:minimize)


In [3]:
### On importe les librairies nécessaires

import pandas as pd
from pandas import DataFrame
import json
from random import randint

import plotly.io as pio
import plotly.express as px


!pip install geopy
from geopy.geocoders import Nominatim

import requests

In [4]:
# On crée un DF avec les nom des 35 villes
Liste_des_villes = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille","Aix en Provence","Avignon","Uzes","Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]
liste_villes_DF = pd.DataFrame(Liste_des_villes, columns=['Villes'])
liste_villes_DF.head(5)

,Villes
0,Mont Saint Michel
1,St Malo
2,Bayeux
3,Le Havre
4,Rouen


## 1 - On cherche a récupérer les coordonnées GPS de chaques villes

In [5]:
##1 - On cherche a récupérer les coordonnées GPS de chaques villes

### On se connecte à l'API
geolocator = Nominatim(user_agent='lucas')
n = geolocator.geocode('Rouen', language='fr')

### On créee la colonne coordonnées puis on la remplit l'adresse complète en utilisant uniquement le nom de la ville.
liste_villes_DF["NomComplet"] = liste_villes_DF["Villes"].apply(geolocator.geocode)

### On utilise l'adresse complète pour requêter la latitude x longitude de chaque ville
liste_villes_DF["Latitude_Ville"] = liste_villes_DF["NomComplet"].apply(lambda x : x.latitude if x != None else None)
liste_villes_DF["Longitude_Ville"] = liste_villes_DF["NomComplet"].apply(lambda x : x.longitude if x != None else None)

liste_villes_DF["averageTemp"] = float(0)

liste_villes_DF.head(5)
liste_villes_DF.dtypes

Villes              object
NomComplet          object
Latitude_Ville     float64
Longitude_Ville    float64
averageTemp        float64
dtype: object

## 2 - On va chercher a requêter l'API openweathermap pour récupérer les données météorologiques des villes ciblées

In [6]:
apikey = 'dc4842f5bea40a49af2c77a5dc34ab91'
def getWeather(lat, lon):
      # Appel API
    url = f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude=current,minutely,hourly,alerts&appid={apikey}&units=metric"
        
    # On stocke la réponse dans une variable
    response = requests.get(url).json()
    
    # On récupère les températures moyennes des prochains jours
    # On voit bien qu'on a 8 réponse au lieu de 7 attendues. Celle du jour est incluse. 
    tempDay0 = response['daily'][0]['temp']['day']
    tempDay1 = response['daily'][1]['temp']['day']
    tempDay2 = response['daily'][2]['temp']['day']
    tempDay3 = response['daily'][3]['temp']['day']
    tempDay4 = response['daily'][4]['temp']['day']
    tempDay5 = response['daily'][5]['temp']['day']
    tempDay6 = response['daily'][6]['temp']['day']
    tempDay7 = response['daily'][7]['temp']['day']
    
    # On fait la moyenne d'entre eux
    average_temp = (tempDay0 + tempDay1 + tempDay2 + tempDay3 + tempDay4 + tempDay5 + tempDay6 + tempDay7) / 8
    return average_temp


#def getWeather(lat, lon):
#    value = randint(0, 100)
#    return value

In [7]:
for idx, row in liste_villes_DF.iterrows():

    # A chaque itération on stocke la valeur des colonnes lat / lon dans des varaiables que l'on passe plus tard en paramètre    
    lat = row['Latitude_Ville']
    lon = row['Longitude_Ville']
    
    # On appelle la fonction getWeather avec tous les paramètres nécessaires
    average_temp = getWeather(lat, lon)
    
    liste_villes_DF.loc[idx, 'averageTemp'] = average_temp
    
    
print(liste_villes_DF)

                          Villes  \
0              Mont Saint Michel   
1                        St Malo   
2                         Bayeux   
3                       Le Havre   
4                          Rouen   
5                          Paris   
6                         Amiens   
7                          Lille   
8                     Strasbourg   
9   Chateau du Haut Koenigsbourg   
10                        Colmar   
11                     Eguisheim   
12                      Besancon   
13                         Dijon   
14                        Annecy   
15                      Grenoble   
16                          Lyon   
17              Gorges du Verdon   
18            Bormes les Mimosas   
19                        Cassis   
20                     Marseille   
21               Aix en Provence   
22                       Avignon   
23                          Uzes   
24                         Nimes   
25                 Aigues Mortes   
26      Saintes Maries de la

In [8]:
# on trie les trucs par ordre décroissant
liste_villes_DF = liste_villes_DF.sort_values(by='averageTemp', axis=0, ascending=False)
liste_villes_DF.head(3)

,Villes,NomComplet,Latitude_Ville,Longitude_Ville,averageTemp
22,Avignon,"(Avignon, Vaucluse, Provence-Alpes-Côte d'Azur...",43.949249,4.805901,27.23125
24,Nimes,"(Nîmes, Gard, Occitanie, France métropolitaine...",43.837425,4.360069,26.60875
23,Uzes,"(Uzès, Nîmes, Gard, Occitanie, France métropol...",44.012128,4.419672,26.43125


In [9]:
liste_villes_DF.to_csv('exportDfVilles')

In [10]:
fig = px.scatter_mapbox(liste_villes_DF, lat=liste_villes_DF['Latitude_Ville'],
                        lon=liste_villes_DF['Longitude_Ville'], size="averageTemp", 
                        color="averageTemp", zoom=3.5, mapbox_style="carto-positron")
fig.show();

ModuleNotFoundError: No module named 'plotly.graph_objs._icicle'